In [1]:
from Chain_Class import Chain
from Create_Chain import * 
from THIS_rungerrychain import *

import geopandas as gpd
import pandas as pd

In [2]:
shp_file='/Users/rebeccawagner/Documents/GitHub/GerryGainMCMC/Data/State Shp Files/AL/Alabama_VTD_District_Intersection.shp'
gpd.read_file(shp_file).head()

,OBJECTID,Join_Count,TARGET_FID,cd_117,VTD,vtd_cd_117,dups,total,one_race,wht_1,...,last,first,middle,race,res_ad_lat,res_ad_lon,incumbent,Shape_Le_1,Shape_Area,geometry
0,1,0,0,01001,01003000001,01003000001_01001,0,357,334,256,...,None,None,None,None,0.0,0.0,0,1.004504,0.024391,"POLYGON ((-87.76515 31.29735, -87.76360 31.297..."
1,2,0,1,01001,01003000002,01003000002_01001,0,251,242,98,...,None,None,None,None,0.0,0.0,0,1.012120,0.022135,"POLYGON ((-87.81299 31.13709, -87.81653 31.131..."
2,3,0,2,01001,01003000003,01003000003_01001,0,558,538,260,...,None,None,None,None,0.0,0.0,0,0.597227,0.010159,"POLYGON ((-87.90042 30.93770, -87.90144 30.937..."
3,4,0,3,01001,01003000004,01003000004_01001,0,1175,1119,821,...,None,None,None,None,0.0,0.0,0,0.641053,0.012414,"POLYGON ((-87.92361 30.93146, -87.92348 30.931..."
4,5,0,4,01001,01003000005,01003000005_01001,0,1632,1578,401,...,None,None,None,None,0.0,0.0,0,0.188839,0.000877,"POLYGON ((-87.77521 30.83709, -87.77556 30.837..."


In [3]:
def split_counter(partition):
    count_splits = 0
    for item in partition["county_info"]:
        count_splits += len(partition["county_info"][item][2]) - 1
    return count_splits

def num_split_counties(partition):
    counties_with_splits = 0
    for item in partition["county_info"]:
        if len(partition["county_info"][item][2]) > 1:
            counties_with_splits += 1
    return counties_with_splits

def oversplit_counties(partition):
    oversplits = 0
    for item in partition["county_info"]:
        if len(partition["county_info"][item][2]) > 2:
            oversplits += 1
    return oversplits

def bad_incumbents(partition):
    num_problems = 0
    for part in partition.parts:
        if partition["incumbents"][part] > 1:
            num_problems += 1
    return num_problems

def subpop_over_threshold(partition):
    count_over = 0
    alpha = 0.5# Arbitrarily defined threshold
    for dist in partition['population']:
        # whatever this first term is needs to be an updater in my updaters so the constraint validator can look for it
        if partition['subpop'][dist] / partition['population'][dist] >= alpha:
            count_over += 1
    return count_over

In [4]:
updaters_list = [split_counter,num_split_counties]

In [5]:
test_chain = CreateChain(steps=10,shp_file=shp_file,grouping_assignment_column='cd_117',updaters_list=updaters_list)

creating chain object..
defining updaters...
creating partition


/Users/rebeccawagner/opt/anaconda3/envs/gerry/lib/python3.10/site-packages/gerrychain/graph/adjacency.py:21: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  geometries[i].id = i
/Users/rebeccawagner/opt/anaconda3/envs/gerry/lib/python3.10/site-packages/gerrychain/graph/adjacency.py:23: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  tree = STRtree(geometries)


chain object with no constraints created


/Users/rebeccawagner/opt/anaconda3/envs/gerry/lib/python3.10/site-packages/gerrychain/graph/graph.py:162: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()
/Users/rebeccawagner/opt/anaconda3/envs/gerry/lib/python3.10/site-packages/gerrychain/graph/graph.py:272: UserWarning: NA values found in column last!
  warnings.warn("NA values found in column {}!".format(column))
/Users/rebeccawagner/opt/anaconda3/envs/gerry/lib/python3.10/site-packages/gerrychain/graph/graph.py:272: UserWarning: NA values found in column first!
  warnings.warn("NA values found in column {}!".format(column))
/Users/rebeccawagner/opt/anaconda3/envs/gerry/lib/python3.10/site-packages/gerrychain/graph/graph.py:272: UserWarning: NA values found in column middle!
  warnings.warn("NA values found in column {}!".format(column))
/Users/rebeccawagner/opt/ana

In [6]:
test_chain.get_current_partition()

<GeographicPartition [7 parts]>

In [8]:
test_chain.get_updaters()

{'split_counter': <function __main__.split_counter(partition)>,
 'num_split_counties': <function __main__.num_split_counties(partition)>,
 'population': <gerrychain.updaters.tally.Tally at 0x165b50d00>,
 'county_info': <function gerrychain.updaters.county_splits.county_splits.<locals>._get_county_splits(partition)>,
 'incumbents': <gerrychain.updaters.tally.Tally at 0x165b50e20>,
 'subpop': <gerrychain.updaters.tally.Tally at 0x165b52b60>}

In [9]:
pop_constraint = constraints.within_percent_of_ideal_population(test_chain.get_current_partition(), test_chain.pop_deviation)

county_limit = constraints.UpperBound(
    lambda p: split_counter(p),
    3*split_counter(test_chain.get_current_partition()))

contiguity = constraints.contiguous

constraints_list=[pop_constraint,county_limit,contiguity]

In [10]:
AddConstraints(chain=test_chain, constraints_list=constraints_list)

0

In [15]:
RunGerryChain(chain_obj=test_chain,seed=10,geographic_area_column='VTD',population_column='total',output_path='/Users/rebeccawagner/Documents/GitHub/GerryGainMCMC/test_output_2',show_progress=True)

Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10

Chain complete. Time elapsed: 1.5643010139465332 seconds.


In [23]:
one = pd.read_csv("/Users/rebeccawagner/Documents/GitHub/GerryGainMCMC/test_folder/test_output")

In [13]:
two = pd.read_csv('/Users/rebeccawagner/Documents/GitHub/GerryGainMCMC/test_folder/test_output_2')

In [14]:
one.equals(two)

True

In [15]:
import os 

In [16]:
os.listdir('/Users/rebeccawagner/Documents/GitHub/GerryGainMCMC/test_folder')[0]

'test_output'

In [24]:
one.set_index('Iteration',inplace=True)

In [29]:
replace = one.iloc[:,-1:]
replace

,10
Iteration,
01003000001,1001
01003000002,1001
01003000003,1001
01003000004,1001
01003000005,1001
...,...
01131005006,1007
01131006001,1007
01131006002,1007


In [40]:
new_gdf = test_chain.gdf.merge(replace,left_on='VTD',right_index=True)
new_gdf.drop(labels='cd_117',axis=1,inplace=True)

In [44]:
new_gdf.rename(columns={"10":'cd_117'},inplace=True)

In [45]:
new_gdf

,OBJECTID,Join_Count,TARGET_FID,VTD,vtd_cd_117,dups,total,one_race,wht_1,blk_1,...,first,middle,race,res_ad_lat,res_ad_lon,incumbent,Shape_Le_1,Shape_Area,geometry,cd_117
0,1,0,0,01003000001,01003000001_01001,0,357,334,256,59,...,None,None,None,0.0,0.0,0,1.004504,0.024391,"POLYGON ((-87.76515 31.29735, -87.76360 31.297...",1001
1,2,0,1,01003000002,01003000002_01001,0,251,242,98,139,...,None,None,None,0.0,0.0,0,1.012120,0.022135,"POLYGON ((-87.81299 31.13709, -87.81653 31.131...",1001
2,3,0,2,01003000003,01003000003_01001,0,558,538,260,270,...,None,None,None,0.0,0.0,0,0.597227,0.010159,"POLYGON ((-87.90042 30.93770, -87.90144 30.937...",1001
3,4,0,3,01003000004,01003000004_01001,0,1175,1119,821,275,...,None,None,None,0.0,0.0,0,0.641053,0.012414,"POLYGON ((-87.92361 30.93146, -87.92348 30.931...",1001
4,5,0,4,01003000005,01003000005_01001,0,1632,1578,401,1148,...,None,None,None,0.0,0.0,0,0.188839,0.000877,"POLYGON ((-87.77521 30.83709, -87.77556 30.837...",1001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1839,1840,0,1839,01131005006,01131005006_01007,0,508,505,39,465,...,None,None,None,0.0,0.0,0,0.141377,0.000727,"POLYGON ((-87.31821 32.02500, -87.31764 32.024...",1007
1840,1841,0,1840,01131006001,01131006001_01007,0,429,420,145,275,...,None,None,None,0.0,0.0,0,0.394519,0.005829,"POLYGON ((-87.55981 32.05385, -87.55962 32.052...",1007
1841,1842,0,1841,01131006002,01131006002_01007,0,1242,1217,480,733,...,None,None,None,0.0,0.0,0,0.819720,0.011736,"POLYGON ((-87.55691 32.02856, -87.55576 32.028...",1007
1842,1843,0,1842,01131006004,01131006004_01007,0,208,208,16,192,...,None,None,None,0.0,0.0,0,0.448974,0.007100,"POLYGON ((-87.62858 31.94270, -87.62749 31.940...",1007
